# Exercise 2.1 - Compilation modes

Let’s examine the differences between nopython mode and object mode. Define a function which is compiled by Numba, and the same function again, with the restriction that it must be compiled on nopython mode:

In [1]:
from numba import jit

@jit
def add(a, b):
    return a + b

@jit(nopython=True)
def add_nopython(a, b):
    return a + b

We can call add with unsupported types:

In [2]:
add("A", "B")

'AB'

Have a look at the typing which Numba generated:

In [3]:
add.inspect_types()

add (str, str)
--------------------------------------------------------------------------------
# File: <ipython-input-1-13f4285e6146>
# --- LINE 3 --- 

@jit

# --- LINE 4 --- 

def add(a, b):

    # --- LINE 5 --- 
    # label 0
    #   a = arg(0, name=a)  :: pyobject
    #   b = arg(1, name=b)  :: pyobject
    #   $0.3 = a + b  :: pyobject
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: pyobject
    #   del $0.3
    #   return $0.4

    return a + b




The `pyobject` type is any Python type - that is, Numba does not know or cannot constrain the types of a variable with this type. As a result, the generated code contains calls to CPython C API functions:

In [5]:
def show_asm(func):
    asm_code = func.inspect_asm()
    for k, v in asm_code.items():
        print('-' * 80)
        print("Signature:", k)
        print('-' * 80)
        print(v)
        print()

show_asm(add)

--------------------------------------------------------------------------------
Signature: (str, str)
--------------------------------------------------------------------------------
	.text
	.file	"<string>"
	.globl	__main__.add$1.pyobject.pyobject
	.align	16, 0x90
	.type	__main__.add$1.pyobject.pyobject,@function
__main__.add$1.pyobject.pyobject:
	.cfi_startproc
	pushq	%rbp
.Ltmp0:
	.cfi_def_cfa_offset 16
	pushq	%r15
.Ltmp1:
	.cfi_def_cfa_offset 24
	pushq	%r14
.Ltmp2:
	.cfi_def_cfa_offset 32
	pushq	%r13
.Ltmp3:
	.cfi_def_cfa_offset 40
	pushq	%r12
.Ltmp4:
	.cfi_def_cfa_offset 48
	pushq	%rbx
.Ltmp5:
	.cfi_def_cfa_offset 56
	pushq	%rax
.Ltmp6:
	.cfi_def_cfa_offset 64
.Ltmp7:
	.cfi_offset %rbx, -56
.Ltmp8:
	.cfi_offset %r12, -48
.Ltmp9:
	.cfi_offset %r13, -40
.Ltmp10:
	.cfi_offset %r14, -32
.Ltmp11:
	.cfi_offset %r15, -24
.Ltmp12:
	.cfi_offset %rbp, -16
	movq	%r8, %r14
	movq	%rcx, %r12
	movq	%rdi, %r15
	testq	%rdx, %rdx
	je	.LBB0_1
	movabsq	$Py_IncRef, %r13
	movq	%r12, %rdi
	callq	*%r13


Notice how there are calls to functions such as `Py_IncRef`, etc. - the need to involve the Python interpreter and reference counting can create a lot of additional overhead.

Using nopython mode avoids these overheads, by making any operation that would require help from the Python interpreter an error. Try:

In [6]:
add_nopython("A", "B")

TypingError: Failed at nopython (nopython frontend)
Invalid usage of + with parameters (str, str)
Known signatures:
 * (int64, int64) -> int64
 * (int64, uint64) -> int64
 * (uint64, int64) -> int64
 * (uint64, uint64) -> uint64
 * (float32, float32) -> float32
 * (float64, float64) -> float64
 * (complex64, complex64) -> complex64
 * (complex128, complex128) -> complex128
 * (uint64,) -> uint64
 * (uint16,) -> uint64
 * (uint8,) -> uint64
 * (uint32,) -> uint64
 * (int32,) -> int64
 * (int8,) -> int64
 * (int16,) -> int64
 * (int64,) -> int64
 * (float32,) -> float32
 * (float64,) -> float64
 * (complex64,) -> complex64
 * (complex128,) -> complex128
 * parameterized
File "<ipython-input-1-13f4285e6146>", line 9

Strings are not supported in nopython mode, resulting in the error. The add_nopython function does work for supported types:

In [7]:
add_nopython(1, 2)

3

## Unification

Not only must all types be resolved to a supported type when using nopython mode, all types must also unify to a single supported type at the time of compilation. For example:

In [8]:
@jit(nopython=True)
def select(a, b, c):
    if c:
        x = a
    else:
        x = b
    return x

select(1, (1, 2), True)

TypingError: Failed at nopython (nopython frontend)
cannot unify int64 and (int64 x 2) for 'x'

Note that the typing cannot depend on the runtime behaviour of the code. In this example, it may be that select is only ever called with `c = True`, so the function would only ever return one type. However, Numba cannot know this, and must always generate typing and code for all paths.

## Unsupported functions

Finally, let’s try calling a function that is not supported in nopython mode:

In [10]:
import numpy as np

@jit(nopython=True)
def scaled_det(A, x):
    return np.linalg.det(A) * x

scaled_det(np.arange(9).reshape(3,3), 2.0)

TypingError: Failed at nopython (nopython frontend)
Unknown attribute 'det' for Module(<module 'numpy.linalg' from '/home/gmarkall/miniconda3/envs/pyconuk34/lib/python3.4/site-packages/numpy/linalg/__init__.py'>) $0.2 $0.3 = getattr(value=$0.2, attr=det)
File "<ipython-input-10-e0f26eceb1c2>", line 5

The above error message indicates that the `det` method is unsupported by Numba.

## Summary

- You can instruct Numba to only use nopython mode by passing `nopython=True` to the jit decorator.
- This doesn't change the way Numba works, but makes it an error if the function can't be compiled in nopython mode.
- Type unification is done without any analysis to determine whether a branch will be taken or not taken - all branches must be able to be unified.
- Calling functions unsupported by Numba will result in an error in nopython mode.